[View in Colaboratory](https://colab.research.google.com/github/gamante91/pytorch_notebooks/blob/master/ConvNN_MNIST.ipynb)

In [2]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' #'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print('Platform:', platform, 'Accelerator:', accelerator)

!pip install --upgrade torch
!pip install --upgrade torchvision

Platform: cp36-cp36m Accelerator: cu80
    100% |████████████████████████████████| 519.5MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58b0e000 @  0x7f9cfba191c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 3.8MB/s 
    100% |████████████████████████████████| 2.0MB 8.1MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', device)

Torch 0.4.1 CUDA 9.0.176
Device: cpu


In [4]:
# Hyper-parameter
batch_size = 100

# Load the MNIST dataset with torchvision 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [0]:
# Hyper-parameters
num_epochs = 5
num_classes = 10
batch_size = 100
alpha = 0.001

In [0]:
# Define Convolutional Neural Network model
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet,self).__init__()
    
    # First Conv Layer
    self.layer1 = nn.Sequential(
      nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,stride=2))
    
    # Second Conv Layer
    self.layer2 = nn.Sequential(
      nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,stride=2))
    
    # Output Layer
    self.fc = nn.Linear(7*7*32, num_classes)
    
    def forward(self,x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.reshape(out.size(0), -1)
      out = self.fc(out)
      return out

In [7]:
# Instanciate the model
model = ConvNet(num_classes).to(device)

print("Model description:\n", model)

Model description:
 ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


In [0]:
# Construct loss function and optimizer
criterion = nn.CrossEntropyLoss()                           # Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=alpha)  # Stochastic Gradient Descent

In [9]:
# Train the model
losses = []

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Compute loss
        loss = criterion(outputs, labels)
    
        # Compute gradient and update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    losses.append(loss.item())
    print("Epoch:", epoch, "loss:", loss.item())

plt.plot(range(num_epochs), losses)
plt.ylabel('Loss')
plt.xlabel('epoch')

NotImplementedError: ignored